In [2]:
import json
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path("../../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
    load_dotenv(str(dotenv_path))


data = json.load(open(Path(os.environ["PROJECT_HOME_DIR"]) / "data/train.json"))

Loaded .env file!


In [3]:
from collections import Counter

labels = []

for d in data:
    labels.extend(d["labels"])

print("Total non 'O'", sum([1 for l in labels if l != "O"]))

cntr = Counter(labels)
cntr

Total non 'O' 2739


Counter({'O': 4989794,
         'B-NAME_STUDENT': 1365,
         'I-NAME_STUDENT': 1096,
         'B-URL_PERSONAL': 110,
         'B-ID_NUM': 78,
         'B-EMAIL': 39,
         'I-STREET_ADDRESS': 20,
         'I-PHONE_NUM': 15,
         'B-USERNAME': 6,
         'B-PHONE_NUM': 6,
         'B-STREET_ADDRESS': 2,
         'I-URL_PERSONAL': 1,
         'I-ID_NUM': 1})

In [4]:
fold_labels = [[] for _ in range(4)]

for d in data:
    idx = d["document"] % 4
    fold_labels[idx].extend(d["labels"])

for f in range(4):
    print(f"Fold {f}")
    print("Total non 'O'", sum([1 for l in fold_labels[f] if l != "O"]))
    cntr = Counter(fold_labels[f])
    print(cntr)

Fold 0
Total non 'O' 658
Counter({'O': 1259432, 'B-NAME_STUDENT': 320, 'I-NAME_STUDENT': 278, 'B-URL_PERSONAL': 25, 'B-ID_NUM': 25, 'B-EMAIL': 8, 'B-USERNAME': 1, 'I-ID_NUM': 1})
Fold 1
Total non 'O' 728
Counter({'O': 1265149, 'B-NAME_STUDENT': 364, 'I-NAME_STUDENT': 282, 'B-URL_PERSONAL': 33, 'B-ID_NUM': 27, 'B-EMAIL': 14, 'I-PHONE_NUM': 6, 'B-PHONE_NUM': 2})
Fold 2
Total non 'O' 738
Counter({'O': 1235353, 'B-NAME_STUDENT': 385, 'I-NAME_STUDENT': 276, 'B-URL_PERSONAL': 27, 'I-STREET_ADDRESS': 20, 'B-ID_NUM': 14, 'B-EMAIL': 8, 'B-USERNAME': 4, 'B-STREET_ADDRESS': 2, 'I-URL_PERSONAL': 1, 'B-PHONE_NUM': 1})
Fold 3
Total non 'O' 615
Counter({'O': 1229860, 'B-NAME_STUDENT': 296, 'I-NAME_STUDENT': 260, 'B-URL_PERSONAL': 25, 'B-ID_NUM': 12, 'B-EMAIL': 9, 'I-PHONE_NUM': 9, 'B-PHONE_NUM': 3, 'B-USERNAME': 1})


In [8]:
num_pii_labels = 0
for k,v in cntr.items():
    if k == "O":
        continue
    num_pii_labels += v

num_pii_labels

2739

### 90% of the labels are names

In [10]:
num_name_student = cntr["B-NAME_STUDENT"] + cntr["I-NAME_STUDENT"]

num_name_student, num_name_student/num_pii_labels

(2461, 0.8985031033223805)

### 4% are urls

In [11]:
num_url_personal = cntr["B-URL_PERSONAL"] + cntr["I-URL_PERSONAL"]

num_url_personal, num_url_personal/num_pii_labels

(111, 0.040525739320920046)

# MOST OF THE FOCUS SHOULD BE ON IDENTIFYING NAMES

but... the deciding factor could be one of the lesser-represented labels. 1 FN to 1 TP on public lb is +0.001